In [1]:

import torch
from torch import nn
import torch.nn.functional as f
import math

In [5]:
x = torch.rand(128, 32, 512)
d_model = 512
n_head = 8

In [4]:
class MultiHeadAttention(nn.Module):
    def _init_(self, d_model, n_head):
        super(MultiHeadAttention, self)._init_()
        self.n_head = n_head
        self.d_model = d_model

        # TODO self.w_q = nn.Linear(d_model, d_model)
        self.w_q = nn.Linear(d_model, n_head)
        self.w_k = nn.Linear(d_model, n_head)
        self.w_v = nn.Linear(d_model, n_head)

        self.w_combine = nn.Linear(d_model, n_head)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, q, k, v, mask=None):
        batch, time, dimension = q.shape
        n_d = self.d_model//self.n_head
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)

        q = q.view(batch, time, self.n_head. n_d).permute(0, 2, 1, 3)
        k = k.view(batch, time, self.n_head. n_d).permute(0, 2, 1, 3)
        v = v.view(batch, time, self.n_head. n_d).permute(0, 2, 1, 3)

        score = q@k.transpose(2,3)/math.sqrt(n_d)

        # 是否使用掩码
        if mask is not None:
            score = score.masked_fill(mask==0, -10000)
        score = self.softmax(score)@v
        # 将加权值重新排列
        score = score.permute(0, 2, 1, 3).contiguous().View(batch, time, dimension)
        output = self.w_combine(score)
        return output

In [6]:
# 实际应用
attention = MultiHeadAttention(d_model, n_head)

TypeError: MultiHeadAttention.__init__() takes 1 positional argument but 3 were given

In [ ]:
out = attention(x, x, x)